# Практическая работа 4
**Тема:** Влияние функций потерь и метрик на качество сегментации  
**Цель:** Научиться применять разные функции потерь и метрики в задаче семантичекой сегментации.
      Исследовать, как они влияют на обучение модели U-Net на основе ResNet

## Подготовка окружения

In [3]:
#!pip install segmentation-models-pytorch torchmetrics

# Загрузка и предобработка данных
## Пример: Oxford Pet Dataset из torchvision

In [6]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["TORCH_USE_CUDA_DSA"]="TRUE"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
import torchvision.transforms as T
import torchvision
import matplotlib.pyplot as plt
from torch import nn

from segmentation_models_pytorch import Unet
from segmentation_models_pytorch.losses import DiceLoss, FocalLoss
from torchmetrics.classification import MulticlassJaccardIndex, MulticlassAccuracy
from torchvision.datasets import OxfordIIITPet
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

# Трансформация изображений к единому размеру
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

target_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.NEAREST),
    transforms.PILToTensor()  # Маска в виде целых чисел
])

dataset = OxfordIIITPet(
    root='./data', 
    download=True, 
    target_types='segmentation', 
    transform=transform, 
    target_transform=target_transform)

train_data, val_data = random_split(dataset, [int(0.8*len(dataset)), int(0.2*len(dataset))])

train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_data, batch_size=4)

## Определение модели

In [11]:
NUM_CLASSES = 3

my_device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
print(f"Используется устройство: {my_device}")

model = Unet(encoder_name="resnet34", in_channels=3, classes= NUM_CLASSES).to(my_device)

Используется устройство: cuda


## Настройка функций потерь

In [14]:
losses = {
    'CrossEntropy': nn.CrossEntropyLoss(),
    'Dice': DiceLoss(mode='multiclass'),
    'Focal': FocalLoss(mode='multiclass')
}

## Метрики

In [17]:
metrics = {
    'IoU': MulticlassJaccardIndex(num_classes=NUM_CLASSES).to(my_device),
    'Accuracy': MulticlassAccuracy(num_classes=NUM_CLASSES).to(my_device)
}

## Цикл обучения

In [20]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0
    for x,y in dataloader:
        x, y = x.to(device), (y.squeeze(1).long()-1).to(device) # уменьшим значения масок на 1, т.к. из датасета они приходят 1-3, а нужно 0-2
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y.long())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    iou_total, acc_total = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), (y.squeeze(1).long()-1).to(device) # уменьшим значения масок на 1, т.к. из датасета они приходят 1-3, а нужно 0-2
            out = model(x)
            preds = torch.argmax(out, dim=1)
            iou_total += metrics['IoU'](preds, y)
            acc_total += metrics['Accuracy'](preds, y)
    return iou_total / len(dataloader), acc_total / len(dataloader)

## Эксперименты

In [23]:
results = []

for name, loss_fn in losses.items():
    print(f"Training with {name} loss...")
    model = Unet(encoder_name="resnet34", in_channels=3, classes= NUM_CLASSES).to(my_device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(10):
        loss = train_one_epoch(model, train_loader, loss_fn, optimizer, my_device)
        iou, acc = evaluate(model, val_loader, my_device)
        print(f"{epoch+1}: Loss={loss:.6f}, IoU={iou:.6f}, Acc={acc:.6f}")

    results.append((name, loss, iou.item(), acc.item()))

Training with CrossEntropy loss...
1: Loss=0.480607, IoU=0.625663, Acc=0.749168
2: Loss=0.375544, IoU=0.639869, Acc=0.743441
3: Loss=0.338006, IoU=0.689777, Acc=0.791109
4: Loss=0.305190, IoU=0.709761, Acc=0.807692
5: Loss=0.279511, IoU=0.690839, Acc=0.784864
6: Loss=0.268344, IoU=0.708460, Acc=0.806453
7: Loss=0.255973, IoU=0.702511, Acc=0.791640
8: Loss=0.245601, IoU=0.734676, Acc=0.826666
9: Loss=0.227229, IoU=0.719565, Acc=0.804379
10: Loss=0.221002, IoU=0.701845, Acc=0.803886
Training with Dice loss...
1: Loss=0.272764, IoU=0.642087, Acc=0.782508
2: Loss=0.228632, IoU=0.673536, Acc=0.793442
3: Loss=0.212248, IoU=0.687766, Acc=0.809715
4: Loss=0.203999, IoU=0.661926, Acc=0.793160
5: Loss=0.197872, IoU=0.670868, Acc=0.790547
6: Loss=0.191927, IoU=0.699978, Acc=0.809622
7: Loss=0.179161, IoU=0.711951, Acc=0.831920
8: Loss=0.175686, IoU=0.704430, Acc=0.818906
9: Loss=0.178799, IoU=0.711474, Acc=0.822059
10: Loss=0.167816, IoU=0.721780, Acc=0.829327
Training with Focal loss...
1: Loss=

## Сравнение результатов

In [51]:
import pandas as pd

print("Результаты обучения модели")
results_df = pd.DataFrame(results, columns=['Loss Function', 'Final Loss', 'Val IoU', 'Val Accuracy'])
print(results_df)

best_idx = 0
best_iou = results[best_idx][2]

for i in range (len(results)):
    iou = results[i][2]
    if iou > best_iou:
        best_idx = i
        best_iou = iou
best_loss = results[best_idx][0]

print(f"Лучшая фукнкиця потерь - {best_loss}. Её IoU - {best_iou:.6f}")

acc_diff = results[1][3] - results[2][3] 

print(f"Разница в Accuracy между Dice и Focal функциями Loss составила - {acc_diff:6f}")

Результаты обучения модели
  Loss Function  Final Loss   Val IoU  Val Accuracy
0  CrossEntropy    0.221002  0.701845      0.803886
1          Dice    0.167816  0.721780      0.829327
2         Focal    0.121513  0.730042      0.824092
Лучшая фукнкиця потерь - Focal. Её IoU - 0.730042
Разница в Accuracy между Dice и Focal функциями Loss составила - 0.005234


## Выводы

1) Лучшая функция потерь - Focal. Она показала лучший результат в значении IoU, хотя не самый улчший в Accuracy
2) Разница в точности (Accuracy) между применением Dice и Focal loss-функций составила порядка 0.005, что не критично в большинстве случаев применения, однако зависит от спицифики предметной области
3) Выбор функции потерь влияет на скорость изменения потерь, а также за их изначальные значения. В результат при применении различных функций потерь можно обнаружить, что при одинаковом кол-ве эпох обучения и скорости обучения, модели на выходе получают результаты, которые могут сильно отличаться (как в случае применения кросс-энтроии и focal из данной работы)
